In [57]:
import pandas as pd
import random
import numpy as np
import warnings

In [ ]:
#Import AS Datasets DAM
#31 days, 24 hours $/MW
#data set includes hourly value, should interpolate to 4

#RD/RU
dfAsRDPrices=pd.read_csv()


In [58]:
dfTripData = pd.read_csv("location_data.csv")
dfLoadData=pd.read_csv("use_data_2wk.csv") #365 days, 1 min increment, 142 houses 
dfGenData=pd.read_csv("annual_capfac_15MinInterpolation.csv") #representative solar day, 15 min interval
dfAsPrices=pd.read_csv('20180701_20180701_PRC_AS_DAM_v1.csv')
dfCostElectric=pd.read_csv("TOU_Rate.csv") #15 min interal 4-9 pm, I think this is tou B

In [59]:
intNumHouseNodes = 5
intNumCommNodes = 2
intTotalNodes = intNumHouseNodes + intNumCommNodes
intAvgNumCarsNode = 5 # goal is to have about 5 * 5 total cars
intSdNumCarsNode = 1
intProfiles = dfTripData.shape[0]

# could make battery parameters a randomized instantiation
fltCarEff = 3.95 # mi/kWh
fltBatteryCap = 40. # kWh
fltChargeRate = 1.9 # kW
fltDt = 0.25 # delta T
fltHomeRate = 7 # kW
fltWorkRate = 7 # kW

# time and conversion intervals
int_minutes=int(15)
int_run_days=2 # day
int_run_hours=24 # hours
int_run_time_interval=60/int_minutes #
time_kwh=15/60  # convert kw data to kwh


# cost to charge
lsCostElectric = []
lsHolder = dfCostElectric.values.tolist()
for ind in range(int_run_days):
    for n in lsHolder:
        lsCostElectric.append(n[0])

In [60]:
def createCarsNodes(intNumHouseNodes, intNumCommNodes, intAvgNumCarsNode, intSdNumCarsNode, \
                    intProfiles, dfTripData):

    '''
    Create nodes car trips and assign work and home nodes
    '''

    #initialize data frame to hold master
    dfNodesTrips = pd.DataFrame(columns=['TripId','ResNode','CommNode','HouseID'])
    house=int(0)
    for intNode in range(intNumHouseNodes):

        # find how many vehicles will be at that residential node
        intNodeNumCars = int(random.gauss(intAvgNumCarsNode,intSdNumCarsNode))
        # assign cars to that node
        lsCarIds = [] # container to hold trip id for that node
        lsCommNode = [] # container to hold where the commerical node
        lsResNode = [] # container to hold the res node
        lsVehicleHouses=[] #container to hold the house related to each vehicle/trip
        for intCar in range(intNodeNumCars):

            # find a car from the data frame
            ind = random.randint(0,intProfiles)
            lsCarIds.append(dfTripData['ID'].iloc[ind]) # add to car id container

            # save res node, added one because 1 initialized index was mismached with comm and load nodes
            lsResNode.append(intNode)

            # find which commercial node that car goes to
            lsCommNode.append(random.randint(0,intNumCommNodes-1))

            #add a house id to associate with the trip id
            lsVehicleHouses.append(house)

            #iterate up house id, 0 based indexing to match the array calls/nodes
            house+=int(1)

        # create data frame and add to master
        dfNode = pd.DataFrame(list(zip(lsCarIds,lsResNode,lsCommNode,lsVehicleHouses)), \
                              columns=['TripId','ResNode','CommNode','HouseID'])

        dfNodesTrips = dfNodesTrips.append(dfNode)

    dfNodesTrips.reset_index(inplace=True, drop=True)

    intVeHouses=house

    return dfNodesTrips,intVeHouses

In [61]:
def getAsValues(dfAsPrices,int_run_days):
    dfAsPricesRu = dfAsPrices.loc[(dfAsPrices['ANC_REGION'] == 'AS_CAISO_EXP') \
                                    & (dfAsPrices['ANC_TYPE'] == 'RU')].sort_values('OPR_HR').reset_index()
    dfAsPricesRd = dfAsPrices.loc[(dfAsPrices['ANC_REGION'] == 'AS_CAISO_EXP') & \
                                  (dfAsPrices['ANC_TYPE'] == 'RD')].sort_values('OPR_HR').reset_index()

    lsRuValue = []
    lsRdValue = []

    for intHr in range(24):

        # get hourly $/MW price -- convert to 15 minute kWh
        fltUp = dfAsPricesRu['MW'].iloc[intHr]/1000
        fltDown = dfAsPricesRd['MW'].iloc[intHr]/1000

        for intInterval in range(4):
            lsRuValue.append(fltUp)
            lsRdValue.append(fltDown)

    # adjust for simulation run period
    for ind in range(int_run_days-1):

        lsRdValue += lsRdValue
        lsRuValue += lsRuValue

    return lsRuValue, lsRdValue


In [77]:
def convertLoadData(dfLoadData,dfNodesTrips,int_run_days,int_run_hours,int_minutes, intVeHouses,intNumCommNodes,intNumHouseNodes):
    '''
    process use data from 1 minute intervals to be on 15 minute interals and for specified run time, and associate with nodes and vehicles
    Each 24 hour time frame goes from 0 (midnight) to 24 (midnight)
    '''
    #initialize time frame to pull
    minutes=60*int_run_hours
    run_length=minutes*int_run_days
    intIntervals=int(run_length/int_minutes)

    #clean dataframe and convert to numpty for manipulation
     #pull only nonzeros columns, but all of them have like one so just set a threshhold
    dfLoadData = dfLoadData.dropna(axis=1,thresh=2)
    npLoadData=dfLoadData.to_numpy()
    run_loads=npLoadData[0:run_length,2:]    # run lenth extracted from dataset, 130+ loads included, skip timestep/day
    run_loads=run_loads.astype("float32")#I think iteration is faster in an np array?
    #Initialize profile length to pull
    intNumLoads = intVeHouses
    nLoads=len(run_loads[0,:])

    #container for house loads
    arrHouseLoads=np.zeros((intNumLoads, intIntervals))  #loadsxtime
    #process to 15 minutes fgor houses
    for n in range(intNumLoads):
        random_select=random.randrange(0,nLoads,1)
        house_load_1min=run_loads[:,random_select] #1 house one vehicle
        o=int(0)
        j=0
        while o < (run_length-int_minutes):
            start=o
            end=o+int_minutes
            with warnings.catch_warnings():
                warnings.filterwarnings('error')
            try:
                new_interval=np.nanmean(house_load_1min[start:end])
            except RuntimeWarning:
                new_interval=0  
            
            arrHouseLoads[n,j]=new_interval
            o+=int_minutes
            j+=1


    #calculate cumulative home nodes
    dfNodeLoads=pd.DataFrame()  #rows will be t, columns will be node ID information, reverse of other df for easy lookup by node
    dctHomeNode=dict() #save node to id mapping, may be useful
    cumulativeLoads=np.zeros((intIntervals))
    i=0 #columns in the node dataframe

    for n in range(intNumHouseNodes):
        #find all the houses that match that Node
        NodeID='ResNode'+str(n)
        lsHomeNode=dfNodesTrips[dfNodesTrips['ResNode']==int(n)].index.tolist()#all home/vehicles that have that resnode associated in dfNodesTrips
        sumrows=arrHouseLoads[lsHomeNode,:]
        cumulativeLoads=np.nansum(sumrows,axis=0)  #this should just be one column
        dctHomeNode[NodeID]=lsHomeNode
        dfNodeLoads.insert(i,NodeID,cumulativeLoads,True)
        i+=1

    arrCommercialLoads=np.zeros((intIntervals,intNumCommNodes))
    #commercial node loads, related to multiple vehicles
    for k in range(intNumCommNodes):  #home x10 for commercial, should replace with real commercial values if possible!
        random_select=random.randrange(0,nLoads,1)
        comm_load_1min=50*run_loads[:,random_select]
        o=int(0)
        j=0
        while o < (run_length-int_minutes):
            start=o
            end=o+int_minutes
            new_interval=np.nanmean(comm_load_1min[start:end])
            arrCommercialLoads[j,k]=new_interval
            o+=int_minutes
            j+=1
        dfNodeLoads.insert(i,"CommNode"+str(k),arrCommercialLoads[:,k],True)
        i+=1

    PeakTimes=dfNodeLoads.idxmax()
    PeakLoads=dfNodeLoads.max()


    lsNodes=dfNodeLoads.columns.values.tolist()
    return   arrHouseLoads,dfNodeLoads, PeakTimes,PeakLoads, lsNodes, dctHomeNode

In [78]:
dfNodesTrips, intVeHouses = createCarsNodes(intNumHouseNodes, intNumCommNodes, intAvgNumCarsNode, intSdNumCarsNode, \
                    intProfiles, dfTripData)

lsRuValue, lsRdValue = getAsValues(dfAsPrices, int_run_days)

arrHouseLoads,dfNodeLoads, PeakTimes,PeakLoads, \
lsNodes,dctHomeNode = convertLoadData(dfLoadData,dfNodesTrips, \
                                    int_run_days,int_run_hours,int_minutes, intVeHouses,intNumCommNodes,intNumHouseNodes)


In [79]:
print(dfNodeLoads)

     ResNode0   ResNode1  ResNode2  ResNode3  ResNode4   CommNode0   CommNode1
0    5.119533   9.202333  5.406467  1.098867  1.277000   24.906670   19.343334
1    5.824133  10.207933  3.551000  1.172467  1.325067   18.253332  141.276672
2    5.252733   9.583200  4.590333  1.665133  1.565933   23.163332   14.136667
3    8.995400   8.138800  3.769867  1.279333  1.441867   22.526667   33.083332
4    7.550934   4.541933  5.109933  1.070933  1.403067   18.393333   22.240000
..        ...        ...       ...       ...       ...         ...         ...
187  6.173067  13.516267  3.271800  2.263267  1.237933   96.493332  396.723328
188  5.101400  12.082399  3.731667  2.916467  1.309467  123.726677  382.080017
189  4.170667  10.580999  2.523933  2.232733  1.182800   68.403328  377.503326
190  5.051667  10.025001  3.892533  2.130467  1.044533   58.230003  385.096680
191  0.000000   0.000000  0.000000  0.000000  0.000000    0.000000    0.000000

[192 rows x 7 columns]
